In [51]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

reqUrl = "https://www.chess.com/players"

headersList = {
 "Accept": "*/*",
 "User-Agent": "Thunder Client (https://www.thunderclient.com)" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

soup = BeautifulSoup(response.content, 'html.parser')

In [53]:
players_list = soup.find('div', class_='post-preview-list-component').find_all('div', class_='post-author-component')

In [ ]:
for soup in players_list:
    try:
        profile_link = soup.select_one('.post-author-avatar')["href"]
    except:
        profile_link = "None"
    try:
        profile_image = soup.select_one('.post-author-thumbnail')['src']
    except:
        profile_image = "None"
    try:
        player_title = soup.select_one('.master-players-chess-title').text
    except:
        player_title = "None"
    try:
        player_name = soup.select_one('.post-author-name').text.strip()
    except:
        player_name = "None"
    try:
        rating_and_rank = soup.select_one('.master-players-world-stats').text.strip().split("|")
    except:
        rating_and_rank = "None"
    try:
        rating = rating_and_rank[0]
    except:
        rating = "None"
    try:
        rank = str(rating_and_rank[-1]).replace("#", "")
    except:
        rank = "None"
    try:
        country = soup.select_one('.post-author-meta').text.strip()
    except:
        country = "None"

    try:
        today_date = datetime.now().strftime('%d-%m-%Y')
    except:
        today_date = "None"

    # Displaying extracted information
    print("Profile Link:", profile_link)
    print("Profile Image:", profile_image)
    print("Player Title:", player_title)
    print("Player Name:", player_name)
    print("Rating", rating)
    print("Rank:", rank)
    print("Country:", country)
    print("DateUpdated: ", today_date)
    print("-----------------------------------")

In [65]:
import psycopg2
import requests
from psycopg2 import sql
from bs4 import BeautifulSoup
from datetime import datetime

dbname = "postgres"
user = ""
password = ""
host = ""
port = 5432
table_name = ""


def connect_to_database(dbname, user, password, host, port):
        # Establish a connection
        conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

        # Create a cursor object to execute SQL queries
        return conn


def push_data(data, conn, table_name):
    insert_query = sql.SQL("""
        INSERT INTO {} 
        (Name, Rank, Rating, Country, Title, ProfileLink, ProfileImage, LastUpdated)
        VALUES 
        (%(Name)s, %(Rank)s, %(Rating)s, %(Country)s, %(Title)s, %(ProfileLink)s, %(ProfileImage)s, %(LastUpdated)s)
    """).format(sql.Identifier(table_name))

    # Create a cursor and execute the insert query
    with conn.cursor() as cursor:
        cursor.execute(insert_query, data)

    # Commit the changes to the database
    conn.commit()

conn = connect_to_database(dbname, user, password, host, port)
def process1(num):

    reqUrl = "https://www.chess.com/players?page={}".format(num)

    headersList = {
    "Accept": "*/*",
    "User-Agent": "Thunder Client (https://www.thunderclient.com)" 
    }

    payload = ""

    response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

    soup = BeautifulSoup(response.content, 'html.parser')

    players_list = soup.find('div', class_='post-preview-list-component').find_all('div', class_='post-author-component')

    for soup in players_list:
        try:
            profile_link = soup.select_one('.post-author-avatar')["href"]
        except:
            profile_link = "None"
        try:
            profile_image = soup.select_one('.post-author-thumbnail')['src']
        except:
            profile_image = "None"
        try:
            player_title = soup.select_one('.master-players-chess-title').text
        except:
            player_title = "None"
        try:
            player_name = soup.select_one('.post-author-name').text.strip()
        except:
            player_name = "None"
        try:
            rating_and_rank = soup.select_one('.master-players-world-stats').text.strip().split("|")
        except:
            rating_and_rank = "None"
        try:
            rating = rating_and_rank[0]
        except:
            rating = "None"
        try:
            rank = str(rating_and_rank[-1]).replace("#", "")
        except:
            rank = "None"
        try:
            country = soup.select_one('.post-author-meta').text.strip()
        except:
            country = "None"

        try:
            today_date = datetime.now().strftime('%d-%m-%Y')
        except:
            today_date = "None"


        data_dict = {
            'Name': player_name,
            'Rank': rank,
            'Rating': rating,
            'Country': country,
            'Title': player_title,
            'ProfileLink':  profile_link,
            'ProfileImage': profile_image,
            'LastUpdated': today_date
            }
        # Assuming conn is your database connection
        push_data(data_dict, conn, table_name)
        
    return len(players_list), reqUrl

In [ ]:
def append_word_to_file(word, filename="output.log"):
    with open(filename, 'a') as file:
        file.write(str(word) + '\n')

for i in range(41, 1200):
    length, url = process1(i)
    append_word_to_file("{} - {} - {}".format(i, length, url))
    print(i)